In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import copy

In [0]:
# Loading the data
train = pd.read_csv('/content/gdrive/My Drive/train.csv')
valid = pd.read_csv('/content/gdrive/My Drive/valid.csv')
test = pd.read_csv('/content/gdrive/My Drive/test_final.csv')

In [0]:
# Get the unique values of the source as well as the target

# Finding out the vocuabulary of the english words
character_english = []
for i in range(len(train)):
  w = train['ENG'][i].split()
  for j in range(len(w)):
    character_english.append(w[j])
for i in range(len(valid)):
  w = valid['ENG'][i].split()
  for j in range(len(w)):
    character_english.append(w[j])
for i in range(len(test)):
  w = test['ENG'][i].split()
  for j in range(len(w)):
    character_english.append(w[j])

print('The number of unique English characters is %d' %len(np.unique(character_english)))  

english_vocab = np.unique(character_english)

# Finding out the vocuabulary of the Hindi words
character_hindi = []
for i in range(len(train)):
  w = train['HIN'][i].split()
  for j in range(len(w)):
    character_hindi.append(w[j])
for i in range(len(valid)):
  w = valid['HIN'][i].split()
  for j in range(len(w)):
    character_hindi.append(w[j])

print('The number of unique Hindi characters is %d' %len(np.unique(character_hindi)))  

hindi_vocab = np.unique(character_hindi)

The number of unique English characters is 43
The number of unique Hindi characters is 83


In [0]:
#Data Preprocessing

CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }

def create_lookup_tables(vocabulary): # vocabulary contains the vocabularies created above
    # make a list of unique words
    #vocab = set(text.split())
    vocab = vocabulary
    # (1)
    # starts with the special tokens
    vocab_to_int = copy.copy(CODES)

    # the index (v_i) will starts from 4 (the 2nd arg in enumerate() specifies the starting index)
    # since vocab_to_int already contains special tokens
    for v_i, v in enumerate(vocab, len(CODES)):
        vocab_to_int[v] = v_i

    # (2)
    int_to_vocab = {v_i: v for v, v_i in vocab_to_int.items()}

    return vocab_to_int, int_to_vocab

In [0]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
        1st, 2nd args: raw string text to be converted
        3rd, 4th args: lookup tables for 1st and 2nd args respectively
    
        return: A tuple of lists (source_id_text, target_id_text) converted
    """
    # empty list of converted words
    source_text_id = []
    target_text_id = []
    
    # make a list of words (extraction)
    source_words = source_text #source_text.split("\n")
    target_words = target_text #target_text.split("\n")
    
    max_source_word_length = max([len(word.split(" ")) for word in source_words])
    max_target_word_length = max([len(word.split(" ")) for word in target_words])
    
    # iterating through each word 
    for i in range(len(source_words)):
        # extract words one by one
        source_word = source_words[i]
        target_word = target_words[i]
        
        # make a list of characters (extraction) from the chosen word
        source_char = source_word.split(" ")
        target_char = target_word.split(" ")
        
        # empty list of converted characters to index in the chosen word
        source_char_id = []
        target_char_id = []
        
        for index, token in enumerate(source_char):
            if (token != ""):
                source_char_id.append(source_vocab_to_int[token])
        
        for index, token in enumerate(target_char):
            if (token != ""):
                target_char_id.append(target_vocab_to_int[token])
                
        # put <EOS> token at the end of the chosen target word
        # this token suggests when to stop creating a sequence
        target_char_id.append(target_vocab_to_int['<EOS>'])
            
        # add each converted word in the final list
        source_text_id.append(source_char_id)
        target_text_id.append(target_char_id)
    
    return source_text_id, target_text_id

In [0]:
# Data Pre-processing
# create lookup tables for English and Hindi data
source_vocab_to_int, source_int_to_vocab = create_lookup_tables(english_vocab)#(source_text)
target_vocab_to_int, target_int_to_vocab = create_lookup_tables(hindi_vocab)#(target_text)

# create list of words whose characters are represented in index
source_text, target_text = text_to_ids(train['ENG'], train['HIN'], create_lookup_tables(english_vocab)[0], create_lookup_tables(hindi_vocab)[0])#(source_text, target_text, source_vocab_to_int, target_vocab_to_int)



In [0]:
source_int_text, target_int_text = source_text, target_text
source_vocab_to_int, target_vocab_to_int = source_vocab_to_int, target_vocab_to_int

In [0]:
#Building the Neural Network

def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len
  
def hyperparam_inputs():
    lr_rate = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return lr_rate, keep_prob  

In [0]:
display_step = 300

epochs = 13
batch_size = 128

rnn_size = 512
num_layers = 2
#num_layers2 = 2
encoding_embedding_size = 256
decoding_embedding_size = 256

learning_rate = 0.001
keep_probability = 0.1

In [0]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :return: Preprocessed target data
    """
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [0]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    #stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob)  for _ in range(num_layers)])
    
    stacked_cells = tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
    
    outputs, state = tf.nn.bidirectional_dynamic_rnn(stacked_cells, stacked_cells, embed, dtype=tf.float32)
    return outputs, state

In [0]:
attention = 0

In [0]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    if attention == 1:
      # Define attention mechanism
      attn_mech = tf.contrib.seq2seq.LuongMonotonicAttention(
      num_units = rnn_size, memory = [batch_size, max_target_sequence_length, rnn_size])
#       memory_sequence_length = target_sequence_length)

      # Define attention cell
      attn_cell = tf.contrib.seq2seq.AttentionWrapper(
      cell = dec_cell, attention_mechanism = attn_mech,
      alignment_history=True)


      decoder = tf.contrib.seq2seq.BasicDecoder(attn_cell, 
                                                helper, 
                                                encoder_state, 
                                                output_layer)

    else:
      decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                helper, 
                                                encoder_state, 
                                                output_layer)

    
    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [0]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    if attention == 1:
      # Define attention mechanism
      attn_mech = tf.contrib.seq2seq.LuongMonotonicAttention(
      num_units = rnn_size, memory = [batch_size, max_target_sequence_length, rnn_size])
#       memory_sequence_length = target_sequence_length)

      # Define attention cell
      attn_cell = tf.contrib.seq2seq.AttentionWrapper(
      cell = dec_cell, attention_mechanism = attn_mech,
      alignment_history=True)


      decoder = tf.contrib.seq2seq.BasicDecoder(attn_cell, 
                                                helper, 
                                                encoder_state, 
                                                output_layer)

    else:
      decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                helper, 
                                                encoder_state, 
                                                output_layer)
    
#     decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
#                                               helper, 
#                                               encoder_state, 
#                                               output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [0]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [0]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_word_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_word_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [0]:
#Build the graph

max_target_word_length = max([len(word) for word in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    lr, keep_prob = hyperparam_inputs()
    
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate shou

In [0]:
#Get the batches and pad the source and target sequences

def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

In [0]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  



In [0]:
epochs = 40
display_step = 10



In [0]:
save_path = '/content/gdrive/My Drive/checkpoints/dev'


with tf.Session(graph=train_graph) as sess:
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        #print('Epoch 1')
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(get_batches(train_source, train_target, batch_size,source_vocab_to_int['<PAD>'],target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})

           
            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

#     test_logits = sess.run(inference_logits, 
#                           {input_data : })
    
    
    
    #Save Model
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/102 - Train Accuracy: 0.7727, Validation Accuracy: 0.7749, Loss: 1.2668
Epoch   0 Batch   20/102 - Train Accuracy: 0.6815, Validation Accuracy: 0.7798, Loss: 1.4506
Epoch   0 Batch   30/102 - Train Accuracy: 0.7279, Validation Accuracy: 0.7817, Loss: 1.1348
Epoch   0 Batch   40/102 - Train Accuracy: 0.6268, Validation Accuracy: 0.7354, Loss: 1.3784
Epoch   0 Batch   50/102 - Train Accuracy: 0.7203, Validation Accuracy: 0.7694, Loss: 1.0353
Epoch   0 Batch   60/102 - Train Accuracy: 0.7329, Validation Accuracy: 0.7150, Loss: 0.8030
Epoch   0 Batch   70/102 - Train Accuracy: 0.7299, Validation Accuracy: 0.7853, Loss: 1.0372
Epoch   0 Batch   80/102 - Train Accuracy: 0.7572, Validation Accuracy: 0.7528, Loss: 0.7760
Epoch   0 Batch   90/102 - Train Accuracy: 0.8040, Validation Accuracy: 0.7861, Loss: 0.7177
Epoch   0 Batch  100/102 - Train Accuracy: 0.6344, Validation Accuracy: 0.7881, Loss: 1.3612
Epoch   1 Batch   10/102 - Train Accuracy: 0.7868, Validation Accuracy

In [0]:
def save_params(params):
    with open('params.p', 'wb') as out_file:
        pickle.dump(params, out_file)


def load_params():
    with open('params.p', mode='rb') as in_file:
        return pickle.load(in_file)

In [0]:
# Save parameters for checkpoint
import pickle
save_params(save_path)


In [0]:
# #Checkpoint

# import tensorflow as tf
# import numpy as np
# import problem_unittests as tests

# _, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

In [0]:
#Transliterate the word
def word_to_seq(word, vocab_to_int):
    results = []
    for char in word.split(" "):
        if char in vocab_to_int:
            results.append(vocab_to_int[char])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

translate_word = "A M E R I C A N"

translate_word = word_to_seq(translate_word, source_vocab_to_int)

# translate_sentence = train['ENG'][0]

# translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

# for i in range(len(train['ENG'])):
#   #translate_sentence = sentence_to_seq(train['ENG'][i], source_vocab_to_int)
#   translate_sentence = train['ENG'][i]
#   if i == 0:
#     translated_sentence = np.vstack([translate_sentence, translate_sentence])
#   else:
#     translated_sentence = np.vstack([translated_sentence, translate_sentence])
  
# translated_sentence = translated_sentence[1:]

In [0]:
translate_word

[16, 28, 20, 33, 24, 18, 16, 29]

In [0]:
loaded_graph = tf.Graph()
counter = 0

with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    str_vec = []
#     for i in range(len(test["ENG"])):
#       str_vec.append(" ")
#     xx = pd.DataFrame({'hin' : str_vec})
    
    for it, s in enumerate(test['ENG']):
      translate_word = word_to_seq(s, source_vocab_to_int)
      
      translate_logits = sess.run(logits, {input_data: [translate_word]*batch_size,
                                         target_sequence_length: [len(translate_word)*2]*batch_size,
                                         keep_prob: 1.0})[0]
      translate_logits = " ".join([target_int_to_vocab[i] for i in translate_logits ])
      
#       xx['hin'][it] = str(translate_logits)
#       if it == 0:
#         final_trans = np.vstack([translate_logits, translate_logits])
#       else:
#         final_trans = np.vstack([final_trans, translate_logits])
      str_vec.append(translate_logits)    
    
# print('Input')
# print('  Word Ids:      {}'.format([i for i in translate_sentence]))
# print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

# print('\nPrediction')
# print('  Word Ids:      {}'.format([i for i in translate_logits]))
# print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/checkpoints/dev


In [0]:
str_vec[1].split()


['आ', 'क', 'श', '<EOS>']

In [0]:
train['HIN'][0].split()

['र', 'ा', 'स', 'व', 'ि', 'ह', 'ा', 'र', 'ी']

In [0]:
str_vec1 = []
for i in str_vec:
  str_vec1.append(i.split('<')[0])

for i in range(len(str_vec1)):
  str_vec1[i] = str_vec1[i][:-1]

In [0]:
str_vec1[0]

'आ ध ि य ा ँ'

In [0]:
loaded_graph = train_graph#tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_word]*batch_size,
                                         target_sequence_length: [len(translate_word)*2]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_word]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_word]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  Hindi Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/checkpoints/dev
Input
  Word Ids:      [41, 24, 30, 29]
  English Words: ['Z', 'I', 'O', 'N']

Prediction
  Word Ids:      [38, 64, 66, 56, 50, 75, 50, 1]
  Hindi Words: ज ़ ि य न ो न <EOS>


In [0]:
def convert(s): 
  
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x  
  
    # return string  
    return new 


In [0]:
yy = copy.deepcopy(str_vec1)
for i in range(len(str_vec1)):
  
  yy[i] = convert(str_vec1[i].split())

  


In [0]:
yy[0]

'आधियाँ'

In [0]:
# pred_s = pd.DataFrame({'hin' : str_vec1})  
pred_s = pd.DataFrame({'id':test['id'], 'HIN': str_vec1}, columns = ['id','HIN'])

In [0]:
pred_s

,id,HIN
0,0,आ ध ि य ा ँ
1,1,आ क श
2,2,आ प ् त ा
3,3,आ य श ा ह
4,4,आ ई
5,5,अ ब ् द ु ह ् ष
6,6,अ ब र र ा थ
7,7,अ ब ी य
8,8,अ ब ू उ न
9,9,अ ब ् र ि य ा


In [0]:
from google.colab import files
pred_s.to_csv('/content/gdrive/My Drive/submission_pa3.csv', index = False)
files.download('/content/gdrive/My Drive/submission_pa3.csv')

In [0]:
 train['HIN'][0]

'र ा स व ि ह ा र ी'